In [19]:
# this file is for device on the client side

# load the dataset

# This file is for trainning
# Run this on the server, or as we called offline. 

import argparse
import base64
import cv2
import datetime
from Models import gatedmodel,mobilenetv2, resnet
import numpy as np
import os
import PIL
import psutil
import sys
import time
import torch
from tqdm import tqdm
from Utils import utils, encoder

middle_sizes_mobile = [1,2,4,8,16]
middle_sizes_resnet = [1,2,4,8,16,32]

middle_sizes = {'mobilenet': middle_sizes_mobile, 'resnet': middle_sizes_resnet}
reduced_sizes = {'cifar-10': (32,32), 'imagenet': (224,224)}
reduced_rates = {'mobilenet': 2, 'resnet': 4}

dataset = 'cifar-10'
model = 'mobilenet'
i_stop = 100

width, height = reduced_sizes[dataset][0]/reduced_rates[model], \
                reduced_sizes[dataset][1]/reduced_rates[model]
middle_size = middle_sizes[model]

# client include client, middle and gate

client = mobilenetv2.mobilenetv2_splitter_client(num_classes = 10, weight_root='./Weights/'+dataset+'/', device='cpu')
# client = resnet.resnet_splitter_client(num_classes=1000, weight_root='./Weights/'+dataset+'/', device='cpu', layers=50)

middle_models = []
for i in range(len(middle_size)):
    middle_models.append(mobilenetv2.MobileNetV2_middle(middle=middle_size[i]))
    # middle_models.append(resnet.resnet_middle(middle=middle_size[i]))

gate_models = []
for i in range(len(middle_size)):
    gate_models.append(gatedmodel.ExitGate(in_planes=middle_size[i],
                                           height = height, width=width))

# eval
client.eval()
for i in range(len(middle_size)):
    middle_models[i].eval()
    gate_models[i].eval()

# quantize
client = torch.ao.quantization.quantize_dynamic(client, {torch.nn.Linear, torch.nn.Conv2d}, dtype=torch.qint8)
for i in range(len(middle_size)):
    middle_models[i] = torch.ao.quantization.quantize_dynamic(middle_models[i], {torch.nn.Linear, torch.nn.Conv2d}, dtype=torch.qint8)
    gate_models[i] = torch.ao.quantization.quantize_dynamic(gate_models[i], {torch.nn.Linear, torch.nn.Conv2d}, dtype=torch.qint8)

# 2. dataset
# directly read bmp image from the storage
data_root = '../data/'+dataset+'-client/'
data_client_out = []
for i in range(len(middle_size)):
    data_client_out.append('../data/'+dataset+'-'+model+'-client-'+str(middle_size[i])+'/')
    if not os.path.exists(data_client_out[i]):
        os.makedirs(data_client_out[i])
    
images_list = os.listdir(data_root)
images_list.remove('labels.txt')
# remove ending with jpg
images_list = [x for x in images_list if x.endswith('.bmp')]
images_list = sorted(images_list)

client_time = [0] * len(middle_size)

# this is test the overspeed, so we don't need to load the models
with torch.no_grad():
        for i, i_path in tqdm(enumerate(images_list)):
            if i >= i_stop:
                break
            
            image_path = data_root + i_path
            image = cv2.imread(image_path, cv2.IMREAD_COLOR)
            image = image.astype(np.float32)/255.0
            image = torch.tensor(image)
            image = image.unsqueeze(0)
            image = image.permute(0, 3, 1, 2)
            
            with torch.no_grad():
                s_time = time.time()
                client_out = client(image).detach()
                for j in range(len(middle_size)):
                    middle_in = middle_models[j].in_layer(client_out)
                    gate_out = gate_models[j](middle_in)
                    middle_in = middle_in.squeeze(0)
                    middle_int = utils.float_to_uint(middle_in)
                    middle_int = middle_int.numpy().copy(order='C')
                    middle_int = middle_int.astype(np.uint8)
                    send_in = base64.b64encode(middle_int)
                    # store it in the folder
                    with open(data_client_out[j] + i_path[:-4], 'wb') as f:
                        f.write(send_in)
                    s1_time = time.time()
                    client_time[j] += s1_time - s_time

for i in range(len(client_time)):
    client_time[i] /= i_stop / 1000 # ms per frame

# print the list without [ and ]
out_string = str(client_time).replace('[','').replace(']','')
print(dataset, model)
print(out_string)


100it [00:00, 226.90it/s]

cifar-10 mobilenet
0.8240985870361328, 1.9571852684020996, 2.882969379425049, 3.5321617126464844, 4.309632778167725


In [22]:
# read a image and store it
# image = cv2.imread('../data/imagenet-client/0.bmp', cv2.IMREAD_COLOR)
# # encode
# image = image.astype(np.uint8)
# image = base64.b64encode(image)
# # store
# with open('../data/imagenet-client/0b', 'wb') as f:
#     f.write(image)

In [28]:
# data path
data_root = '../data/imagenet-client/'
# get '.bmp'
import os 
images_list = os.listdir(data_root)
images_list = [x for x in images_list if x.endswith('.bmp')]

data_out = '../data/imagenet-jpeg/'

import cv2
# compress the image jpeg
for i_path in images_list:
    image_path = data_root + i_path
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    for quality in range(5, 100, 5):
        cv2.imwrite(data_out+i_path[:-4]+'_'+str(quality)+'.jpg', image, [int(cv2.IMWRITE_JPEG_QUALITY), quality])


In [108]:
# load embbeded 
dataset = 'cifar-10'
model = 'mobilenet'

from Models import mobilenetv2, resnet
model = mobilenetv2.MobileNetV2(num_classes=10)
model.load_state_dict(torch.load('./Weights/'+dataset+'/pretrained/mobilenetv2.pth'))
model = model.to('cuda')
model.eval()

jpeg25_path = '../data/'+dataset+'-jpeg25/'
root_path = '../data/'+dataset+'-client/'
labels = root_path+'labels.txt'
labels = open(labels, 'r')
labels = labels.readlines()
labels = [x[:-1] for x in labels]
labels = torch.tensor([int(x) for x in labels])
labels

images = [str(x) for x in range(600)]
preds = []

from torchvision import transforms
if dataset == 'cifar-10':
    normal = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])
elif dataset == 'imagenet':
    normal = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

with torch.no_grad():
    server_time = []
    for image in images:
        
        image_path = jpeg25_path + image
        i_read = open(image_path, 'rb')
        image = i_read.read()
        # decode
        s_time = time.time()
        image = base64.b64decode(image)
        image = np.frombuffer(image, dtype=np.uint8)
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
        image = normal(image)
        image = image.unsqueeze(0)
        image = image.to('cuda')
        pred = model(image)
        pred = torch.argmax(pred, dim=1)
        preds.append(pred.item())
        e_time = time.time()
        server_time.append(e_time - s_time)

preds = torch.tensor(preds)
accuracy = [0] * 10 
server_time_box = [0] * 10
# calculate the accuracy every 60 frames
for i in range(0, 600, 60):
    accuracy[i//60] = torch.sum(preds[i:i+60] == labels[i:i+60]).item() / 60
    server_time_box[i//60] = sum(server_time[i:i+60]) / 60*1000

print(jpeg25_path)
print(accuracy)
print(server_time_box)

../data/cifar-10-jpeg25/
[0.65, 0.7, 0.6333333333333333, 0.6833333333333333, 0.65, 0.7, 0.7166666666666667, 0.6666666666666666, 0.7666666666666667, 0.7166666666666667]
[3.6353707313537598, 3.5940567652384443, 3.534579277038574, 3.5523414611816406, 3.596754868825277, 3.6237597465515137, 3.4986217816670733, 3.618709246317546, 3.582139809926351, 3.4907301266988116]


In [109]:
# load embbeded 
dataset = 'cifar-10'
model = 'mobilenet'

from Models import mobilenetv2, resnet
model = mobilenetv2.MobileNetV2(num_classes=10)
model.load_state_dict(torch.load('./Weights/'+dataset+'/pretrained/mobilenetv2.pth'))
model = model.to('cuda')
model.eval()

jpeg75_path = '../data/'+dataset+'-jpeg75/'
root_path = '../data/'+dataset+'-client/'
labels = root_path+'labels.txt'
labels = open(labels, 'r')
labels = labels.readlines()
labels = [x[:-1] for x in labels]
labels = torch.tensor([int(x) for x in labels])
labels

images = [str(x) for x in range(600)]
preds = []

from torchvision import transforms
if dataset == 'cifar-10':
    normal = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])
elif dataset == 'imagenet':
    normal = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

with torch.no_grad():
    server_time = []
    for image in images:
        
        image_path = jpeg75_path + image
        i_read = open(image_path, 'rb')
        image = i_read.read()
        # decode
        s_time = time.time()
        image = base64.b64decode(image)
        image = np.frombuffer(image, dtype=np.uint8)
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
        image = normal(image)
        image = image.unsqueeze(0)
        image = image.to('cuda')
        pred = model(image)
        pred = torch.argmax(pred, dim=1)
        preds.append(pred.item())
        e_time = time.time()
        server_time.append(e_time - s_time)

preds = torch.tensor(preds)
accuracy = [0] * 10 
server_time_box = [0] * 10
# calculate the accuracy every 60 frames
for i in range(0, 600, 60):
    accuracy[i//60] = torch.sum(preds[i:i+60] == labels[i:i+60]).item() / 60
    server_time_box[i//60] = sum(server_time[i:i+60]) / 60*1000

print(jpeg75_path)
print(accuracy)
print(server_time_box)

../data/cifar-10-jpeg75/
[0.8, 0.8666666666666667, 0.7666666666666667, 0.7833333333333333, 0.7166666666666667, 0.7666666666666667, 0.8333333333333334, 0.7666666666666667, 0.9, 0.8833333333333333]
[3.3351898193359375, 3.51340373357137, 3.528730074564616, 3.3399899800618487, 3.288976351420085, 3.2774964968363447, 3.330787022908529, 3.3434510231018066, 3.3651749293009443, 3.3732573191324873]


In [110]:
# load embbeded 
dataset = 'cifar-10'
model = 'mobilenet'

from Models import mobilenetv2, resnet
model = mobilenetv2.MobileNetV2(num_classes=10)
model.load_state_dict(torch.load('./Weights/'+dataset+'/pretrained/mobilenetv2.pth'))
model = model.to('cuda')
model.eval()

cjpeg_path = '../data/'+dataset+'-cjpeg/'
root_path = '../data/'+dataset+'-client/'
labels = root_path+'labels.txt'
labels = open(labels, 'r')
labels = labels.readlines()
labels = [x[:-1] for x in labels]
labels = torch.tensor([int(x) for x in labels])
labels

images = [str(x) for x in range(600)]
preds = []

from torchvision import transforms
if dataset == 'cifar-10':
    normal = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])
elif dataset == 'imagenet':
    normal = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

with torch.no_grad():
    server_time = []
    for image in images:
        
        image_path = cjpeg_path + image
        i_read = open(image_path, 'rb')
        image = i_read.read()
        # decode
        s_time = time.time()
        image = base64.b64decode(image)
        image = np.frombuffer(image, dtype=np.uint8)
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
        image = normal(image)
        image = image.unsqueeze(0)
        image = image.to('cuda')
        pred = model(image)
        pred = torch.argmax(pred, dim=1)
        preds.append(pred.item())
        e_time = time.time()
        server_time.append(e_time - s_time)

preds = torch.tensor(preds)
accuracy = [0] * 10 
server_time_box = [0] * 10
# calculate the accuracy every 60 frames
for i in range(0, 600, 60):
    accuracy[i//60] = torch.sum(preds[i:i+60] == labels[i:i+60]).item() / 60
    server_time_box[i//60] = sum(server_time[i:i+60]) / 60*1000

print(cjpeg_path)
print(accuracy)
print(server_time_box)

../data/cifar-10-cjpeg/
[0.8166666666666667, 0.8, 0.7666666666666667, 0.8166666666666667, 0.7166666666666667, 0.8, 0.8333333333333334, 0.7833333333333333, 0.8666666666666667, 0.8666666666666667]
[3.3003171284993487, 3.4371654192606607, 3.4213980038960776, 3.306754430135091, 3.404005368550618, 3.368230660756429, 3.4621795018513994, 3.341809908548991, 3.330520788828532, 3.3794204394022627]


In [115]:
# load embbeded 
dataset = 'cifar-10'
model = 'mobilenet'

from Models import mobilenetv2, resnet
model = mobilenetv2.MobileNetV2(num_classes=10)
model.load_state_dict(torch.load('./Weights/'+dataset+'/pretrained/mobilenetv2.pth'))
model = model.to('cuda')
model.eval()

cjpeg_path = '../data/'+dataset+'-cjpeg-ML/'
root_path = '../data/'+dataset+'-client/'
labels = root_path+'labels.txt'
labels = open(labels, 'r')
labels = labels.readlines()
labels = [x[:-1] for x in labels]
labels = torch.tensor([int(x) for x in labels])
labels

images = [str(x) for x in range(600)]
preds = []

from torchvision import transforms
if dataset == 'cifar-10':
    normal = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])
elif dataset == 'imagenet':
    normal = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

with torch.no_grad():
    server_time = []
    for image in images:
        
        image_path = cjpeg_path + image
        # check is the path exist
        i_read = open(image_path, 'rb')
        image = i_read.read()
        
        # decode
        s_time = time.time()
        image = base64.b64decode(image)
        image = np.frombuffer(image, dtype=np.uint8)
        print(len(image))
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
        print(image)
        image = normal(image)
        image = image.unsqueeze(0)
        image = image.to('cuda')
        pred = model(image)
        pred = torch.argmax(pred, dim=1)
        preds.append(pred.item())
        e_time = time.time()
        server_time.append(e_time - s_time)

preds = torch.tensor(preds)
accuracy = [0] * 10 
server_time_box = [0] * 10
# calculate the accuracy every 60 frames
for i in range(0, 600, 60):
    accuracy[i//60] = torch.sum(preds[i:i+60] == labels[i:i+60]).item() / 60
    server_time_box[i//60] = sum(server_time[i:i+60]) / 60*1000

print(cjpeg_path)
print(accuracy)
print(server_time_box)

[173 113  53 ...  15  69 116]
None


TypeError: pic should be PIL Image or ndarray. Got <class 'NoneType'>

In [103]:
# load embbeded 
dataset = 'cifar-10'
model = 'mobilenet'

from Models import mobilenetv2, resnet
model = mobilenetv2.MobileNetV2(num_classes=10)
model.load_state_dict(torch.load('./Weights/'+dataset+'/pretrained/mobilenetv2.pth'))
model = model.to('cuda')
model.eval()

original_path = '../data/'+dataset+'-client/'
root_path = '../data/'+dataset+'-client/'
labels = root_path+'labels.txt'
labels = open(labels, 'r')
labels = labels.readlines()
labels = [x[:-1] for x in labels]
labels = torch.tensor([int(x) for x in labels])

images = [str(x)+'.bmp' for x in range(600)]
preds = []

from torchvision import transforms
if dataset == 'cifar-10':
    normal = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])
elif dataset == 'imagenet':
    normal = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
from PIL import Image
with torch.no_grad():
    for i, image in enumerate(images):
        image_path = original_path + image
        i_read = Image.open(image_path).convert('RGB')
        image = np.array(i_read)
        image = image.astype(np.float32)/255.0
        # print(image)
        image = normal(image)
        image = image.unsqueeze(0)
        image = image.to('cuda')
        
        pred = model(image)
        pred = torch.argmax(pred, dim=1)
        preds.append(pred.item())

preds = torch.tensor(preds)
accuracy = [0] * 10 
# calculate the accuracy every 60 frames
for i in range(0, 600, 60):
    accuracy[i//60] = torch.sum(preds[i:i+60] == labels[i:i+60]).item() / 60

print(accuracy)

[0.8666666666666667, 0.85, 0.7833333333333333, 0.8166666666666667, 0.7333333333333333, 0.8333333333333334, 0.8666666666666667, 0.8, 0.8833333333333333, 0.8666666666666667]


In [96]:
from Dataloaders import dataloader_cifar10
_, test, _ = dataloader_cifar10.Dataloader_cifar10_val(test_batch=1)
for i, (image, label) in enumerate(test):
    image = image.to('cuda')
    pred = model(image)
    pred = torch.argmax(pred, dim=1)

Files already downloaded and verified
Files already downloaded and verified
tensor([[-6.8665, -2.0951, -7.5659,  2.3243, -5.8989,  0.6056, -3.0840, -3.9598,
         -4.0189, -4.1408]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([3], device='cuda:0')
tensor([[ -1.7792,   0.8832, -11.1968, -10.5257, -11.8882, -11.5903, -13.3459,
         -13.7443,   5.9686,  -4.3372]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor([8], device='cuda:0')
tensor([[  0.8940,  -2.6838,  -5.0191,  -7.0503,  -5.6024,  -7.5521, -11.0815,
          -6.9351,   4.1427,  -4.1215]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor([8], device='cuda:0')
tensor([[ 3.8037, -2.8443, -2.3276, -5.0625, -8.1910, -7.3822, -6.0312, -7.6261,
         -2.6992, -3.6173]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([0], device='cuda:0')
tensor([[ -9.6393,  -8.6659,  -4.3395,  -2.3219,  -1.1398,  -8.2178,   5.5698,
          -7.3277, -12.7354, -11.7017]], device='cuda:0',
       grad_fn=<AddmmBack